In [1]:
import torch
import numpy as np
import pynq
from pynq import Overlay

from pynq import MMIO
from pynq import allocate
from pynq import Interrupt
import asyncio
import sys
import time
import math
import csv 

In [2]:
# Configure the hardware 
overlay = Overlay('feather.bit') # 
top_ip = overlay.QCONV_0
top_ip.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, RESERVED_3=0, RESERVED_4=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED=0),
  ifc1_offset_1 = Register(ifc1_offset=write-only),
  ifc1_offset_2 = Register(ifc1_offset=write-only),
  ifc2_offset_1 = Register(ifc2_offset=write-only),
  ifc2_offset_2 = Register(ifc2_offset=write-only),
  ifc3_offset_1 = Register(ifc3_offset=write-only),
  ifc3_offset_2 = Register(ifc3_offset=write-only),
  ifc4_offset_1 = Register(ifc4_offset=write-only),
  ifc4_offset_2 = Register(ifc4_offset=write-only),
  ifc5_offset_1 = Register(ifc5_offset=write-only),
  ifc5_offset_2 = Register(ifc5_offset=write-only),
  ifc6_offset_1 = Register(ifc6_offset=write-only),
  ifc6_offset_2 = Register(ifc6_offset=write-only),
  ifc7_1 = Register(ifc7=write-only),
  ifc7_2 = Regis

In [3]:
layer_idx_target_subnet_to_supernet = [[0, 1, 3, 4, 5, 6, 7, 8, 9, 16, 17, 18, 19, 20, 21, 22, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 48, 49, 50, 51, 52, 53, 54],
[0, 1, 3, 4, 5, 6, 7, 8, 9, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57],
[0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57],
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60],
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60],
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60]]


# top parameters
DEBUG_MODE = False
INITIAL_OUTPUT_BUFFER = False 
MAX_NUM_LAYER = 0
for i in range(len(layer_idx_target_subnet_to_supernet)):
    num_layer_workload = len(layer_idx_target_subnet_to_supernet[i]) 
    MAX_NUM_LAYER = num_layer_workload if (num_layer_workload > MAX_NUM_LAYER) else MAX_NUM_LAYER
    
PRINT_UNIFIED_OUTPUT_BUFFER = True
PRINT_UNIFIED_IN_BUFFER = False
PRINT_COMPILED_DATA = False
PRINT_RECEIVED_OACTS = False

TOTAL_NUM_MODEL = 6
PARPLLELISM = [16, 9, 1] # K, C, Y

# Read the external file from outside data.

In [4]:
def convert_fix32_8_into_int8(scale_fp, output_size):
    decimal_bitwidth_fixed_point = 24
    out_scale_fp_scale_up_int = np.zeros(output_size, dtype="int8")
    size_in = len(scale_fp)
    for data_idx in range((output_size>>2)):
        temp_scale_int = np.round(scale_fp[data_idx] * (2**decimal_bitwidth_fixed_point)) if(data_idx < size_in) else 0
        temp_scale_int = int(temp_scale_int)

        out_scale_fp_scale_up_int[(data_idx<<2)]   = np.array(temp_scale_int).astype(np.int8)       #temp_scale_int        & 0xff
        out_scale_fp_scale_up_int[(data_idx<<2)+1] = np.array(temp_scale_int >> 8).astype(np.int8)  #& 0xff
        out_scale_fp_scale_up_int[(data_idx<<2)+2] = np.array(temp_scale_int >> 16).astype(np.int8) #& 0xff
        out_scale_fp_scale_up_int[(data_idx<<2)+3] = np.array(temp_scale_int >> 24).astype(np.int8) #& 0xff

    return out_scale_fp_scale_up_int

def padding_int8_array_with_specific_zp(int8_data, output_size, specified_zero=None):
    """ 
        default output data type is int32, so every int32 demands 4 int8 
    """
    padding_zero = specified_zero if(specified_zero!=None) else 0 

    delta_size = output_size - int8_data.size
    if delta_size == 0:
        return int8_data
    else:
        int8_padding_data = np.full(delta_size, specified_zero, dtype="int8")
        int8_output_data = np.concatenate((int8_data, int8_padding_data), axis=0)
        return int8_output_data

class QCONV():
    """
        The overall iacts, weights are all stored inside separate buffer.
    """
    
    def __init__ (self, csv_data_path, bitstream_path, parallelism, top_ip):
        # Fuction control knob
        self.FIX_T_K_16 = True
        
        # FPGA Kernel
        self.top_ip = top_ip

        # Internal Parameters
        self.NUM_BYTE_HW_IFC = 16
        self.NUMBER_IACT_DATA_PER_UB_ENTRY = 16
        self.TRIPLE_NUM_HP_IFC = 18 # 3 cycle transfers with 6 interfaces being transferred every cycle.
        self.NUM_BYTE_IACT = 1
        self.NUM_BYTE_WEIGHT = 1
        self.NUM_IACTS = 16
        self.NUM_IACTS_PER_OFF_CHIP_IFC = math.ceil(self.NUM_BYTE_HW_IFC / self.NUM_BYTE_IACT)
        self.NUM_WEIGHTS_PER_OFF_CHIP_IFC = math.ceil(self.NUM_BYTE_HW_IFC / self.NUM_BYTE_WEIGHT)
        self.LOG_PAR_NUM_IACTS = 4
        self.NUMBER_WEIGHTS_DATA_PER_UB_ENTRY = 9

        # External Arguments
        self.PARALLEL_K = parallelism[0] # 16
        self.PARALLEL_C = parallelism[1] # 9
        self.PARALLEL_Y = parallelism[2] # 1
        self.iacts = None
        self.weights = None
        self.oacts = None
        self.overall_scale = None
        self.weights_zp = None
        self.reordered_iacts = None
        self.expected_oacts = None
        self.merged_expected_oacts = None

        self.csv_data_path = csv_data_path
        self.bitstream_path = bitstream_path
        self.num_reordered_iacts = 0
        self.padding_x = 0
        self.padding_y = 0
        self.stride_x = 0
        self.stride_y = 0
        self.dilation_x = 0
        self.dilation_y = 0
        self.iacts_zp_int = 0
        self.iacts_scale = 0
        self.oacts_zp_int = 0
        self.oacts_scale = 0
        self.N = 0
        self.K = 0
        self.C = 0
        self.X = 0
        self.Y = 0
        self.R = 0
        self.S = 0
        self.O_X = 0
        self.O_Y = 0
        
        self.mimic_N = None
        self.mimic_K = None
        self.mimic_C = None
        self.mimic_X = None
        self.mimic_Y = None
        self.mimic_R = None
        self.mimic_S = None
        self.mimic_O_X = None
        self.mimic_O_Y = None
        
        self.num_overall_scale_after_padding_hw_ifc = 0
        self.num_weights_zp_after_padding_hw_ifc = 0
        self.num_weights_after_padding_hw_ifc = 0
        self.num_iacts_after_padding_hw_ifc = 0
        self.unified_in_buffer = None
        self.unified_out_buffer = None

        self.num_reordered_iacts = None
        self.num_iacts_ifc_entries = None
        self.reordered_iacts = None

        self.total_compiled_data = None

        self.MAX_KERNEL_BUF_ENTRY_HALF = 32768
        
    def software_compile_fpga(self):
        self.top_ip.register_map.N = int(self.N)
        self.top_ip.register_map.K = int(self.K)
        self.top_ip.register_map.C = int(self.C)
        self.top_ip.register_map.R = int(self.R)
        self.top_ip.register_map.S = int(self.S)
        self.top_ip.register_map.X = int(self.X)
        self.top_ip.register_map.Y = int(self.Y)
        self.top_ip.register_map.padding_x = int(self.padding_x)
        self.top_ip.register_map.padding_y = int(self.padding_y)
        self.top_ip.register_map.stride_x = int(self.stride_x)
        self.top_ip.register_map.stride_y = int(self.stride_y)
        self.top_ip.register_map.dilation_x = int(self.dilation_x)
        self.top_ip.register_map.dilation_y = int(self.dilation_y)

        self.top_ip.write(0xcc, int(self.iacts_zp_int))
        self.top_ip.write(0xd4, int(self.oacts_zp_int))

        init_addr_oacts = self.num_overall_scale_after_padding_hw_ifc + self.num_weights_zp_after_padding_hw_ifc + self.num_weights_after_padding_hw_ifc + self.num_iacts_after_padding_hw_ifc
        num_k_ceiling_parallel = math.ceil(self.K / self.PARALLEL_K)
        num_iter_oacts = num_k_ceiling_parallel*self.O_X*self.O_Y

        self.unified_in_buffer = pynq.allocate((init_addr_oacts << 4), "int8")
        print(f"number in buffer entries = {(init_addr_oacts << 4)}, dtype={self.unified_in_buffer.dtype}")
        
        self.unified_out_buffer = pynq.allocate((num_iter_oacts << 4), dtype=np.uint8)
        print(f"number output buffer entries = {(num_iter_oacts << 4)}")
        
        self.top_ip.write(0xdc, (0))

        self.unified_in_buffer[:] = self.total_compiled_data[:]
        
        if PRINT_UNIFIED_IN_BUFFER:
            for _idx in range(self.unified_in_buffer.size):
                print(self.unified_in_buffer[_idx])
        
        self.unified_in_buffer.flush()
    
        self.top_ip.write(0x10, self.unified_in_buffer.physical_address & 0xFFFFFFFF )
        self.top_ip.write(0x14, self.unified_in_buffer.physical_address >> 32)
        self.top_ip.write(0x1c, self.unified_in_buffer.physical_address & 0xFFFFFFFF )
        self.top_ip.write(0x20, self.unified_in_buffer.physical_address >> 32)
        self.top_ip.write(0x28, self.unified_in_buffer.physical_address & 0xFFFFFFFF )
        self.top_ip.write(0x2c, self.unified_in_buffer.physical_address >> 32)
        self.top_ip.write(0x34, self.unified_in_buffer.physical_address & 0xFFFFFFFF )
        self.top_ip.write(0x38, self.unified_in_buffer.physical_address >> 32)
        self.top_ip.write(0x40, self.unified_in_buffer.physical_address & 0xFFFFFFFF )
        self.top_ip.write(0x44, self.unified_in_buffer.physical_address >> 32)
        self.top_ip.write(0x4c, self.unified_in_buffer.physical_address & 0xFFFFFFFF )
        self.top_ip.write(0x50, self.unified_in_buffer.physical_address >> 32)
        self.top_ip.write(0x58, self.unified_out_buffer.physical_address & 0xFFFFFFFF )
        self.top_ip.write(0x5c, self.unified_out_buffer.physical_address >> 32)
        
    def run_fpga_profile(self):
        total_number_testing = 200
        start_time = time.time()
        for i in range(total_number_testing):
            self.top_ip.register_map.CTRL.AP_START = 1

            # get it start
            while(int(self.top_ip.register_map.CTRL.AP_IDLE) == 0):
                pass

        end_time = time.time()
        return (end_time - start_time)/total_number_testing
    
    def data_read_from_external_file(self):
        csv_parameters_file = open(f"{self.csv_data_path}/parameter.csv", "r") 
        csv_parameters_data = csv.reader(csv_parameters_file, delimiter="," )
        
        iacts_file = open(f"{self.csv_data_path}/iacts.csv", "r") 
        iacts_data = csv.reader(iacts_file, delimiter="," )

        weights_file = open(f"{self.csv_data_path}/weights.csv", "r") 
        weights_data = csv.reader(weights_file, delimiter="," )
        
        oacts_file = open(f"{self.csv_data_path}/oacts.csv", "r")
        oacts_data = csv.reader(oacts_file, delimiter="," )
        
        weights_parameter_file = open(f"{self.csv_data_path}/weights_quant_parameters.csv", "r")
        weights_parameter_data = csv.reader(weights_parameter_file, delimiter="," )
        
        for row in csv_parameters_data:
            self.padding_x = int(row[0])
            self.padding_y = int(row[1])
            self.stride_x = int(row[2])
            self.stride_y = int(row[3])
            self.dilation_x = int(row[4])
            self.dilation_y = int(row[5])
            self.iacts_zp_int = int(row[6])
            self.iacts_scale = float(row[7])
            self.oacts_zp_int = int(row[8])
            self.oacts_scale = float(row[9])
            self.N = int(row[10])
            self.K = int(row[11])
            self.C = int(row[12])
            self.X = int(row[13])
            self.Y = int(row[14])
            self.R = int(row[15])
            self.S = int(row[16])
            self.O_X = int(row[17])
            self.O_Y = int(row[18])

        print(f"padding_x={self.padding_x}")
        print(f"padding_y={self.padding_y}")
        print(f"stride_x={self.stride_x}")
        print(f"stride_y={self.stride_y}")
        print(f"dilation_x={self.dilation_x}")
        print(f"dilation_y={self.dilation_y}")
        print(f"iacts_zp_int={self.iacts_zp_int}")
        print(f"iacts_scale={self.iacts_scale}")
        print(f"oacts_zp_int={self.oacts_zp_int}")
        print(f"oacts_scale={self.oacts_scale}")
        print(f"N={self.N}")
        print(f"K={self.K}")
        print(f"C={self.C}")
        print(f"X={self.X}")
        print(f"Y={self.Y}")
        print(f"R={self.R}")
        print(f"S={self.S}")
        print(f"O_X={self.O_X}")
        print(f"O_Y={self.O_Y}")

        self.iacts = np.zeros([self.N*self.C*self.X*self.Y], dtype=np.uint8)
        self.weights = np.zeros([self.K*self.C*self.R*self.S], dtype=np.int8)
        self.oacts = np.zeros([self.N*self.K*self.O_X*self.O_Y], dtype=np.uint8)
        self.weights_zp = np.zeros([self.K], dtype=np.int8)
        self.overall_scale = np.zeros([self.K], dtype=np.float64)

        addr_idx = 0
        for row in iacts_data:
            for data in row:
                self.iacts[addr_idx] = int(data)
                addr_idx += 1

        addr_idx = 0
        for row in weights_data:
            for data in row:
                self.weights[addr_idx] = int(data)
                addr_idx += 1

        addr_idx = 0
        for row in oacts_data:
            for data in row:
                self.oacts[addr_idx] = int(data)
                addr_idx += 1

        for row in weights_parameter_data:
            for data_idx in range(len(row)>>1):
                self.weights_zp[data_idx] = int(row[data_idx])
                self.overall_scale[data_idx] = float(row[data_idx + (len(row)>>1)])
                
        print("finish external data processing")
                
    def data_fed_directly_from_pytorch(self, iacts, weights, stride, padding, dilation, output_scale, output_zp):
        self.padding_x = padding[0]
        self.padding_y = padding[1]
        self.stride_x = stride[0]
        self.stride_y = stride[1]
        self.dilation_x = dilation[0]
        self.dilation_y = dilation[1]
        self.iacts_zp_int = iacts.q_zero_point()
        self.iacts_scale = iacts.q_scale()
        self.oacts_zp_int = output_zp
        self.oacts_scale = output_scale
        self.N = iacts.shape[0]
        self.K = weights.shape[0]
        self.C = weights.shape[1]
        self.X = iacts.shape[2]
        self.Y = iacts.shape[3]
        self.R = weights.shape[2]
        self.S = weights.shape[3]
        self.O_X = math.floor((iacts.shape[2] + 2 * padding[0] - dilation[0] * (weights.shape[2] - 1) - 1) / stride[0] + 1)
        self.O_Y = math.floor((iacts.shape[3] + 2 * padding[1] - dilation[1] * (weights.shape[3] - 1) - 1) / stride[1] + 1)
        self.iacts = iacts.int_repr().detach().numpy().flatten()
        self.weights_zp = weights.q_per_channel_zero_points().numpy()
        self.weights_scale = weights.q_per_channel_scales().numpy()
        self.weights = weights.int_repr().detach().numpy().flatten()

        print(f"padding_x={self.padding_x}")
        print(f"padding_y={self.padding_y}")
        print(f"stride_x={self.stride_x}")
        print(f"stride_y={self.stride_y}")
        print(f"dilation_x={self.dilation_x}")
        print(f"dilation_y={self.dilation_y}")
        print(f"iacts_zp_int={self.iacts_zp_int}")
        print(f"iacts_scale={self.iacts_scale}")
        print(f"oacts_zp_int={self.oacts_zp_int}")
        print(f"oacts_scale={self.oacts_scale}")
        print(f"N={self.N}")
        print(f"K={self.K}")
        print(f"C={self.C}")
        print(f"X={self.X}")
        print(f"Y={self.Y}")
        print(f"R={self.R}")
        print(f"S={self.S}")
        print(f"O_X={self.O_X}")
        print(f"O_Y={self.O_Y}")
        
        self.overall_scale = np.zeros(self.K, dtype=np.float32)
        for k in range(self.K):
            self.overall_scale[k] = np.divide(np.multiply(self.weights_scale[k], self.iacts_scale), self.oacts_scale)
        
    def reorder_iacts(self):
        ##############################################
        ### --- Input reorder to satisfy parallelism 
        ##############################################
        # need to r input to satisfy the parallelism requirement of the hardware
        PARALLEL_C = self.PARALLEL_C
        num_c_ceiling = math.ceil(self.C/PARALLEL_C)
        self.num_reordered_iacts = math.ceil(self.C/PARALLEL_C)*self.N*self.X*self.Y*PARALLEL_C
        self.num_iacts_ifc_entries = math.ceil(self.num_reordered_iacts/ self.NUM_IACTS_PER_OFF_CHIP_IFC)
        self.reordered_iacts = np.zeros(self.num_reordered_iacts, dtype=np.uint8)
        for n in range(self.N):
            for c in range(0, self.C, PARALLEL_C):
                for x in range(self.X):
                    for y in range(self.Y):
                        for cp_id in range(PARALLEL_C):
                            self.reordered_iacts[n*num_c_ceiling*self.X*self.Y*PARALLEL_C + int(c/PARALLEL_C)*self.X*self.Y*PARALLEL_C + x*self.Y*PARALLEL_C + y*PARALLEL_C + cp_id] = self.iacts[n*self.C*self.X*self.Y + (c + cp_id)*self.X*self.Y + x*self.Y + y] if (c + cp_id < self.C) else self.iacts_zp_int
        
    def print_in_data_info(self):
        print("overall scales are")
        for data_idx in range(len(self.overall_scale)):
            print(self.overall_scale[data_idx], end=" ")
        print()

        print("weights zero points are")
        for data_idx in range(len(self.weights_zp)):
            print(self.weights_zp[data_idx], end=" ")
        print()
        
        print("iacts data are")
        offset_n = self.C*self.X*self.Y
        offset_x = self.X*self.Y
        offset_y = self.Y
        for n in range(self.N):
            for c in range(self.C):
                for x in range(self.X):
                    for y in range(self.Y):
                        print(self.iacts[n*offset_n + c*offset_x + x*offset_y + y], end=" ")
                    print()
        print()

        print("weights data are")
        offset_weight_k = self.C*self.R*self.S
        offset_weight_c = self.R*self.S
        offset_weight_r = self.S
        for k in range(self.K):
            for c in range(self.C):
                for r in range(self.R):
                    for s in range(self.S):
                        print(self.weights[k*offset_weight_k + c*offset_weight_c + r*offset_weight_r + s], end=" ")
                    print()
        print()

        print("reordered iacts data are")
        offset_n = math.ceil(self.C / self.PARALLEL_C) * self.X * self.Y * self.PARALLEL_C
        offset_c = math.ceil(self.X * self.Y * self.PARALLEL_C)
        offset_x = math.ceil(self.Y * self.PARALLEL_C)
        if self.R == 3:
            for n in range(self.N):
                for c in range(0, self.C, self.PARALLEL_C):
                    for x in range(self.X):
                        for y in range(self.Y):
                            for cp_id in range(self.PARALLEL_C):
                                print(self.reordered_iacts[n*offset_n + c*offset_c + x*offset_x + y*self.PARALLEL_C + cp_id], end=" ")
                            print()
            print()
        else:
            for n in range(self.mimic_N):
                for c in range(0, self.mimic_C, self.PARALLEL_C):
                    for x in range(self.mimic_X):
                        for y in range(self.mimic_Y):
                            for cp_id in range(self.PARALLEL_C):
                                print(self.reordered_iacts_1x1[n*offset_n + c*offset_c + x*offset_x + y*self.PARALLEL_C + cp_id], end=" ")
                            print()
            print()

    def compile_data_feeding_hw(self):
        """
            This subfunction is for the 3x3 kernel only.
        """
        ##############################################
        ### --- using int32 as the transfer data
        ##############################################
        # padding zero to fill in all spaces in 128-bit interface.
        self.num_overall_scale_after_padding_hw_ifc =  math.ceil(len(self.overall_scale) / (self.NUM_BYTE_HW_IFC >> 2))
        self.num_weights_zp_after_padding_hw_ifc = math.ceil(len(self.weights_zp) /  self.NUM_BYTE_HW_IFC)
        self.num_weights_after_padding_hw_ifc = math.ceil(len(self.weights) / self.NUM_BYTE_HW_IFC)
        self.num_iacts_after_padding_hw_ifc = math.ceil(math.ceil(self.num_reordered_iacts / self.NUM_BYTE_HW_IFC) / self.TRIPLE_NUM_HP_IFC) * self.TRIPLE_NUM_HP_IFC

        print(f"len(overall_scale) = {len(self.overall_scale)}  number after padding 128-bit interface and parallel interfaces = {self.num_overall_scale_after_padding_hw_ifc}")
        print(f"len(weights_zp) = {len(self.weights_zp)}  number after padding 128-bit interface and parallel interfaces = {self.num_weights_zp_after_padding_hw_ifc}")
        print(f"len(weights) = {len(self.weights)}  number after padding 128-bit interface and parallel interfaces = {self.num_weights_after_padding_hw_ifc}")
        print(f"len(iacts) = {len(self.reordered_iacts)}  number after padding 128-bit interface and parallel interfaces = {self.num_iacts_after_padding_hw_ifc}")

        ### Allocate space for new input data
        compiled_data_overall_scale = convert_fix32_8_into_int8(self.overall_scale, (self.num_overall_scale_after_padding_hw_ifc << 4))
        compiled_data_weights_zp = padding_int8_array_with_specific_zp(self.weights_zp, (self.num_weights_zp_after_padding_hw_ifc << 4), self.weights_zp[0])
        compiled_data_iacts = padding_int8_array_with_specific_zp(self.reordered_iacts, (self.num_iacts_after_padding_hw_ifc << 4), self.iacts_zp_int)

        # calculate zero entries for weights padding.
        if self.FIX_T_K_16:
            T_K = 16
            total_iteration = math.ceil(self.K/ T_K)
        else:
            num_ub_entry_kernel                  = math.ceil((self.num_weights_after_padding_hw_ifc << 4)/ self.NUMBER_WEIGHTS_DATA_PER_UB_ENTRY)
            total_iteration                      = math.ceil(num_ub_entry_kernel / self.MAX_KERNEL_BUF_ENTRY_HALF)
            T_K_pre                              = math.ceil(self.K / total_iteration)
            T_K                                  = math.ceil(T_K_pre / self.PARALLEL_K) * self.PARALLEL_K

        new_weights_len = (T_K * total_iteration) * self.C * self.R * self.S
        print(f"self.K = {self.K}, T_K = {T_K}, overall_K post padding = {T_K * total_iteration}, total_iteration = {total_iteration}, overall weights length post-padding = {new_weights_len}")
        self.num_weights_after_padding_hw_ifc = math.ceil(new_weights_len / self.NUM_BYTE_HW_IFC)

        compiled_data_weights = padding_int8_array_with_specific_zp(self.weights, (self.num_weights_after_padding_hw_ifc << 4), self.weights_zp[0])

        self.total_compiled_data = np.concatenate((compiled_data_overall_scale, compiled_data_weights_zp, compiled_data_iacts, compiled_data_weights), axis=0)
        print(f"total_compiled_data.size = {self.total_compiled_data.size}")
        if PRINT_COMPILED_DATA:
            for _idx in range(self.total_compiled_data.size):
                print(self.total_compiled_data[_idx])

    def run_workload_on_fpga(self):
        """
            For 3x3 kernel only.
        """
        self.reorder_iacts()
        # qconv2d_instance.print_in_data_info()
        self.compile_data_feeding_hw()
        print("finish compile_data_feeding_hw")
        self.software_compile_fpga()
        print("finish software_compile_fpga")
        latency = self.run_fpga_profile()
        print(f"latency = {latency}") 
        
        return latency

    def self_result_parser(self):
        self.unified_out_buffer.sync_from_device()
        self.unified_out_buffer.flush()
        if PRINT_RECEIVED_OACTS:
            for _idx in range(len(self.unified_out_buffer)):
                print(self.unified_out_buffer[_idx], end=" ")
        
        self.unified_out_buffer = self.unified_out_buffer.reshape([math.ceil(self.K/self.PARALLEL_K), self.O_X, self.O_Y, self.PARALLEL_K])
        self.unified_out_buffer = np.swapaxes(self.unified_out_buffer,2,3)
        self.unified_out_buffer = np.swapaxes(self.unified_out_buffer,1,2)
        self.unified_out_buffer = self.unified_out_buffer.reshape([math.ceil(self.K/self.PARALLEL_K)*self.PARALLEL_K, self.O_X, self.O_Y])
        self.unified_out_buffer = self.unified_out_buffer[0:self.K, :, :]
        print(f"finish reshape")

    def result_parser(self, unified_out_buffer):
        unified_out_buffer.sync_from_device()
        unified_out_buffer.flush()
        unified_out_buffer = unified_out_buffer.reshape([math.ceil(self.K/self.PARALLEL_K), self.O_X, self.O_Y, self.PARALLEL_K])
        unified_out_buffer = np.swapaxes(unified_out_buffer,2,3)
        unified_out_buffer = np.swapaxes(unified_out_buffer,1,2)
        unified_out_buffer = unified_out_buffer.reshape([math.ceil(self.K/self.PARALLEL_K)*self.PARALLEL_K, self.O_X, self.O_Y])
        unified_out_buffer = unified_out_buffer[0:self.K, :, :]
        return unified_out_buffer

# The end to end deployment and testing

In [5]:
latency_table_layer = np.zeros((TOTAL_NUM_MODEL, MAX_NUM_LAYER), dtype=float)

for network_idx in range(TOTAL_NUM_MODEL):
    TOTAL_NUM_LAYER_WORKLOAD = len(layer_idx_target_subnet_to_supernet[network_idx])
    for layer_idx in range(TOTAL_NUM_LAYER_WORKLOAD):
    # for layer_idx in range(TOTAL_NUM_LAYER_WORKLOAD-init_file_id):
        print(f"executing network {network_idx} layer {layer_idx}")
        path_iacts = f"/home/xilinx/jupyter_notebooks/feather/layer_data_weight_sharing_resnet50/conv_iActs_{network_idx}_{layer_idx}.pth"
        path_weights = f"/home/xilinx/jupyter_notebooks/feather/layer_data_weight_sharing_resnet50/conv_weights_{network_idx}_{layer_idx}.pth"
        path_parameters = f"/home/xilinx/jupyter_notebooks/feather/layer_data_weight_sharing_resnet50/conv_parameter_{network_idx}_{layer_idx}.pth"

        iacts = torch.load(path_iacts)
        weights = torch.load(path_weights)
        parameters = torch.load(path_parameters)

        [padding0, padding1, stride0, stride1, dilation0, dilation1, output_scale, output_zp] = parameters

        padding = [int(padding0), int(padding1)]
        stride = [int(stride0), int(stride1)]
        dilation = [int(dilation0), int(dilation1)]
        output_scale = float(output_scale)
        output_zp = int(output_zp)

        print(f"{iacts.shape[0]}, {weights.shape[0]}, {weights.shape[1]}, {weights.shape[2]}, {weights.shape[3]}, {iacts.shape[2]}, {iacts.shape[3]}, {padding[0]}, {padding[1]}, {stride[0]}, {stride[1]}, {dilation[0]}, {dilation[1]}, {output_zp}, {output_scale}")
        o_x = math.floor((iacts.shape[2] + 2*padding[0] - dilation[0] * (weights.shape[2] - 1) - 1) / stride[0] + 1)
        o_y = math.floor((iacts.shape[3] + 2*padding[1] - dilation[1] * (weights.shape[3] - 1) - 1) / stride[1] + 1)

        """
            The workload might overflow the size of unified buffer.
        """
        if weights.shape[2]==1:
            latency_table_layer[network_idx, layer_idx] = 0
        else:
            csv_data_path = ""
            bitstream_path = ""
            qconv2d_instance = QCONV(csv_data_path, bitstream_path, PARPLLELISM, top_ip)
            qconv2d_instance.data_fed_directly_from_pytorch(iacts, weights, stride, padding, dilation, output_scale, output_zp)
            fpga_latency = qconv2d_instance.run_workload_on_fpga()
            qconv2d_instance.self_result_parser()
            latency_table_layer[network_idx, layer_idx] = fpga_latency

print("#### latency of all layers in different models###")
for i in range(latency_table_layer.shape[0]):
    for j in range(latency_table_layer.shape[1]):
        print(latency_table_layer[i,j], end=" ")
    print()
    
latency_table = np.sum(latency_table_layer, axis=1)
print("\n#### latency of different model###\n")
for i in range(latency_table.shape[0]):
    print(latency_table[i])

executing network 0 layer 0
1, 24, 3, 3, 3, 160, 160, 1, 1, 2, 2, 1, 1, 64, 0.4009031057357788
padding_x=1
padding_y=1
stride_x=2
stride_y=2
dilation_x=1
dilation_y=1
iacts_zp_int=57
iacts_scale=0.03740893304347992
oacts_zp_int=64
oacts_scale=0.4009031057357788
N=1
K=24
C=3
X=160
Y=160
R=3
S=3
O_X=80
O_Y=80


/usr/local/share/pynq-venv/lib/python3.10/site-packages/torch/_utils.py:314: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  device=storage.device,


len(overall_scale) = 24  number after padding 128-bit interface and parallel interfaces = 6
len(weights_zp) = 24  number after padding 128-bit interface and parallel interfaces = 2
len(weights) = 648  number after padding 128-bit interface and parallel interfaces = 41
len(iacts) = 230400  number after padding 128-bit interface and parallel interfaces = 14400
self.K = 24, T_K = 16, overall_K post padding = 32, total_iteration = 2, overall weights length post-padding = 864
total_compiled_data.size = 231392
finish compile_data_feeding_hw
number in buffer entries = 231392, dtype=int8
number output buffer entries = 204800
finish software_compile_fpga
latency = 0.0007030272483825684
finish reshape
executing network 0 layer 1
1, 40, 24, 3, 3, 80, 80, 1, 1, 1, 1, 1, 1, 69, 1.4578040838241577
padding_x=1
padding_y=1
stride_x=1
stride_y=1
dilation_x=1
dilation_y=1
iacts_zp_int=61
iacts_scale=0.5724054574966431
oacts_zp_int=69
oacts_scale=1.4578040838241577
N=1
K=40
C=24
X=80
Y=80
R=3
S=3
O_X=80


self.K = 136, T_K = 16, overall_K post padding = 144, total_iteration = 9, overall weights length post-padding = 176256
total_compiled_data.size = 191344
finish compile_data_feeding_hw
number in buffer entries = 191344, dtype=int8
number output buffer entries = 14400
finish software_compile_fpga
latency = 0.0003065764904022217
finish reshape
executing network 0 layer 22
1, 664, 136, 1, 1, 10, 10, 0, 0, 1, 1, 1, 1, 67, 0.11305078864097595
executing network 0 layer 23
1, 136, 664, 1, 1, 10, 10, 0, 0, 1, 1, 1, 1, 58, 0.4851232171058655
executing network 0 layer 24
1, 136, 136, 3, 3, 10, 10, 1, 1, 1, 1, 1, 1, 72, 0.35597026348114014
padding_x=1
padding_y=1
stride_x=1
stride_y=1
dilation_x=1
dilation_y=1
iacts_zp_int=59
iacts_scale=0.4423370361328125
oacts_zp_int=72
oacts_scale=0.35597026348114014
N=1
K=136
C=136
X=10
Y=10
R=3
S=3
O_X=10
O_Y=10
len(overall_scale) = 136  number after padding 128-bit interface and parallel interfaces = 34
len(weights_zp) = 136  number after padding 128-bit in

finish reshape
executing network 1 layer 2
1, 32, 64, 1, 1, 40, 40, 0, 0, 1, 1, 1, 1, 57, 0.9004822969436646
executing network 1 layer 3
1, 32, 32, 3, 3, 40, 40, 1, 1, 1, 1, 1, 1, 60, 0.3900472819805145
padding_x=1
padding_y=1
stride_x=1
stride_y=1
dilation_x=1
dilation_y=1
iacts_zp_int=85
iacts_scale=0.3303258717060089
oacts_zp_int=60
oacts_scale=0.3900472819805145
N=1
K=32
C=32
X=40
Y=40
R=3
S=3
O_X=40
O_Y=40
len(overall_scale) = 32  number after padding 128-bit interface and parallel interfaces = 8
len(weights_zp) = 32  number after padding 128-bit interface and parallel interfaces = 2
len(weights) = 9216  number after padding 128-bit interface and parallel interfaces = 576
len(iacts) = 57600  number after padding 128-bit interface and parallel interfaces = 3600
self.K = 32, T_K = 16, overall_K post padding = 32, total_iteration = 2, overall weights length post-padding = 9216
total_compiled_data.size = 66976
finish compile_data_feeding_hw
number in buffer entries = 66976, dtype=int8

...

len(overall_scale) = 208  number after padding 128-bit interface and parallel interfaces = 52
len(weights_zp) = 208  number after padding 128-bit interface and parallel interfaces = 13
len(weights) = 389376  number after padding 128-bit interface and parallel interfaces = 24336
len(iacts) = 42336  number after padding 128-bit interface and parallel interfaces = 2646
self.K = 208, T_K = 16, overall_K post padding = 208, total_iteration = 13, overall weights length post-padding = 389376
total_compiled_data.size = 432752
finish compile_data_feeding_hw
number in buffer entries = 432752, dtype=int8
number output buffer entries = 40768
finish software_compile_fpga
latency = 0.0009082400798797607
finish reshape
executing network 4 layer 35
1, 1024, 208, 1, 1, 14, 14, 0, 0, 1, 1, 1, 1, 64, 0.13072636723518372
executing network 4 layer 36
1, 360, 1024, 1, 1, 14, 14, 0, 0, 1, 1, 1, 1, 56, 0.37961170077323914
executing network 4 layer 37
1, 360, 360, 3, 3, 14, 14, 1, 1, 1, 1, 1, 1, 74, 0.33262285

len(overall_scale) = 408  number after padding 128-bit interface and parallel interfaces = 102
len(weights_zp) = 408  number after padding 128-bit interface and parallel interfaces = 26
len(weights) = 1498176  number after padding 128-bit interface and parallel interfaces = 93636
len(iacts) = 20286  number after padding 128-bit interface and parallel interfaces = 1278
self.K = 408, T_K = 16, overall_K post padding = 416, total_iteration = 26, overall weights length post-padding = 1527552
total_compiled_data.size = 1550048
finish compile_data_feeding_hw
number in buffer entries = 1550048, dtype=int8
number output buffer entries = 20384
finish software_compile_fpga
latency = 0.0011946427822113036
finish reshape
executing network 4 layer 57
1, 1640, 408, 1, 1, 7, 7, 0, 0, 1, 1, 1, 1, 65, 0.047892987728118896
executing network 4 layer 58
1, 408, 1640, 1, 1, 7, 7, 0, 0, 1, 1, 1, 1, 57, 0.4521695673465729
executing network 4 layer 59
1, 408, 408, 3, 3, 7, 7, 1, 1, 1, 1, 1, 1, 84, 0.138789728

1, 56, 56, 3, 3, 56, 56, 1, 1, 1, 1, 1, 1, 76, 0.05248451977968216
padding_x=1
padding_y=1
stride_x=1
stride_y=1
dilation_x=1
dilation_y=1
iacts_zp_int=69
iacts_scale=0.1320289820432663
oacts_zp_int=76
oacts_scale=0.05248451977968216
N=1
K=56
C=56
X=56
Y=56
R=3
S=3
O_X=56
O_Y=56
len(overall_scale) = 56  number after padding 128-bit interface and parallel interfaces = 14
len(weights_zp) = 56  number after padding 128-bit interface and parallel interfaces = 4
len(weights) = 28224  number after padding 128-bit interface and parallel interfaces = 1764
len(iacts) = 197568  number after padding 128-bit interface and parallel interfaces = 12348
self.K = 56, T_K = 16, overall_K post padding = 64, total_iteration = 4, overall weights length post-padding = 32256
total_compiled_data.size = 230112
finish compile_data_feeding_hw
number in buffer entries = 230112, dtype=int8
number output buffer entries = 200704
finish software_compile_fpga
latency = 0.0011175310611724854
finish reshape
executing ne

latency = 0.0013254296779632569
finish reshape
executing network 5 layer 35
1, 1024, 256, 1, 1, 14, 14, 0, 0, 1, 1, 1, 1, 62, 0.13825148344039917
executing network 5 layer 36
1, 256, 1024, 1, 1, 14, 14, 0, 0, 1, 1, 1, 1, 54, 0.3934932351112366
executing network 5 layer 37
1, 256, 256, 3, 3, 14, 14, 1, 1, 1, 1, 1, 1, 71, 0.34429866075515747
padding_x=1
padding_y=1
stride_x=1
stride_y=1
dilation_x=1
dilation_y=1
iacts_zp_int=62
iacts_scale=0.3059164881706238
oacts_zp_int=71
oacts_scale=0.34429866075515747
N=1
K=256
C=256
X=14
Y=14
R=3
S=3
O_X=14
O_Y=14
len(overall_scale) = 256  number after padding 128-bit interface and parallel interfaces = 64
len(weights_zp) = 256  number after padding 128-bit interface and parallel interfaces = 16
len(weights) = 589824  number after padding 128-bit interface and parallel interfaces = 36864
len(iacts) = 51156  number after padding 128-bit interface and parallel interfaces = 3204
self.K = 256, T_K = 16, overall_K post padding = 256, total_iteration = 16

latency = 0.0034163224697113036
finish reshape
executing network 5 layer 57
1, 2048, 720, 1, 1, 7, 7, 0, 0, 1, 1, 1, 1, 65, 0.04963643476366997
executing network 5 layer 58
1, 408, 2048, 1, 1, 7, 7, 0, 0, 1, 1, 1, 1, 51, 0.5014353394508362
executing network 5 layer 59
1, 408, 408, 3, 3, 7, 7, 1, 1, 1, 1, 1, 1, 85, 0.13954678177833557
padding_x=1
padding_y=1
stride_x=1
stride_y=1
dilation_x=1
dilation_y=1
iacts_zp_int=84
iacts_scale=0.40362945199012756
oacts_zp_int=85
oacts_scale=0.13954678177833557
N=1
K=408
C=408
X=7
Y=7
R=3
S=3
O_X=7
O_Y=7
len(overall_scale) = 408  number after padding 128-bit interface and parallel interfaces = 102
len(weights_zp) = 408  number after padding 128-bit interface and parallel interfaces = 26
len(weights) = 1498176  number after padding 128-bit interface and parallel interfaces = 93636
len(iacts) = 20286  number after padding 128-bit interface and parallel interfaces = 1278
self.K = 408, T_K = 16, overall_K post padding = 416, total_iteration = 26, overa

In [9]:
print(f"Show per-layer latency for all convolution 3x3 layers in {TOTAL_NUM_MODEL} models, the first array dimension={TOTAL_NUM_MODEL}, indicating 6 models; While the second array dimension={MAX_NUM_LAYER}, indiciating the maximal number of layers in all {TOTAL_NUM_MODEL} models, where 0 means the layer has different kernel weights than 3x3.")
print(latency_table_layer)

Show per-layer latency for all convolution 3x3 layers in 6 models, the first array dimension=6, indicating 6 models; While the second array dimension=61, indiciating the maximal number of layers in all 6 models.
[[0.00070303 0.0007663  0.         0.00024622 0.         0.
  0.         0.00024666 0.         0.         0.00069423 0.
  0.         0.         0.00024477 0.         0.         0.00079987
  0.         0.         0.         0.00030658 0.         0.
  0.0003083  0.         0.         0.00039827 0.         0.
  0.00030523 0.         0.         0.0012739  0.         0.
  0.         0.00090901 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.        ]
 [0.00070616 0.00096405 0.         0.00024804 0.         0.
  0.         0.00037131 0.         0.         0.0006993  0.
  0.         0.         0.00037678 0.         

In [8]:
print(f"Show accumulated latency for all layers of {TOTAL_NUM_MODEL} models")
latency_table = np.sum(latency_table_layer, axis=1)

for i in range(latency_table.shape[0]):
    print(latency_table[i])

Show accumulated latency for all layers of 6 models
0.007202377319335938
0.009927009344100953
0.018090410232543947
0.0245594322681427
0.03485630631446838
0.03562654018402099
